In [2]:
import tensorflow as tf

from tensorflow.keras import layers

import numpy as np
import matplotlib.pyplot as plt

#%pip install git+https://github.com/tensorflow/docs

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

%load_ext tensorboard

2025-03-06 15:50:43.903973: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-06 15:50:43.907946: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-06 15:50:43.919860: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741287043.939947    9916 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741287043.946710    9916 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-06 15:50:43.966852: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [3]:
seq = "RRRPPPSSS" * 10
seq

'RRRPPPSSSRRRPPPSSSRRRPPPSSSRRRPPPSSSRRRPPPSSSRRRPPPSSSRRRPPPSSSRRRPPPSSSRRRPPPSSSRRRPPPSSS'

In [4]:
vocab = sorted(set(seq))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return [char2idx[c] for c in text]

def int_to_text(ints):
  return ''.join(idx2char[ints])

In [5]:
length = 20
data = []

for i in range(len(seq) - length):
    data.append(seq[i : length + i + 1])
    
print(len(data), len(data[0]), data[0])

70 21 RRRPPPSSSRRRPPPSSSRRR


In [6]:
encoded_data = np.array([text_to_int(d) for d in data])
print(encoded_data.shape, len(encoded_data), len(encoded_data[0]), encoded_data[0])

(70, 21) 70 21 [1 1 1 0 0 0 2 2 2 1 1 1 0 0 0 2 2 2 1 1 1]


In [7]:
features = encoded_data[:,:-1]
labels = encoded_data[:,-1]
features.shape, labels.shape

((70, 20), (70,))

In [8]:
features_onehot = np.array([tf.keras.utils.to_categorical(feature, num_classes=len(vocab)) for feature in features])
labels_onehot = tf.keras.utils.to_categorical(labels, num_classes=len(vocab))
features_onehot.shape, labels_onehot.shape

((70, 20, 3), (70, 3))

In [64]:
model = tf.keras.Sequential([
    layers.LSTM(128, input_shape=(features_onehot.shape[1], features_onehot.shape[2])),
    layers.Dense(127, activation='relu'),
    layers.Dense(3, activation='softmax')
])

model.compile(loss='CategoricalCrossentropy',
              optimizer='adam',
              metrics=['Accuracy'])

history = model.fit(
    features_onehot,
    labels_onehot,
    epochs=100)

Epoch 1/100


/home/andreati/Coding/Learning/TensorFlow/.venvTF/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - Accuracy: 0.5266 - loss: 1.0919
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - Accuracy: 0.6820 - loss: 1.0477
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - Accuracy: 0.7646 - loss: 1.0118
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - Accuracy: 0.8166 - loss: 0.9705
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - Accuracy: 0.8401 - loss: 0.9221
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - Accuracy: 0.6873 - loss: 0.8716
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - Accuracy: 0.6834 - loss: 0.8011
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - Accuracy: 0.8960 - loss: 0.7157
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - Accuracy: 1.0000 - loss: 0.6053
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - Accuracy: 1.0000 - loss: 0.3966
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - Accuracy: 1.0000 - loss: 0.1802
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - Accuracy: 1.0000 - loss: 0.0577
E

In [111]:
def predict(seq):
    seq_encoded = np.array([text_to_int(seq)]) #encode sequence of chars into sequence of ints
    seq_onehot = np.array([tf.keras.utils.to_categorical(se, num_classes=len(vocab)) for se in seq_encoded]) #encode sequence of ints into sequence of one-hots
    pred = model.predict(seq_onehot) #run the prediction. Return list of probabilities for next char
    idx = np.argmax(pred) #idx of the char with max probability, equal to the integer encoding for the chars
    return int_to_text(idx), round(float(pred[0][idx]), 2)

predict('RRRPPPSSSSS')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


('P', 1.0)